In [2]:
!pip install transformers torch accelerate datasets

In [6]:
import os
token = os.getenv("HF_TOKEN")

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import Dataset
import torch

model_name = "meta-llama/Llama-3.2-1B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, token=token).to("cpu")

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

texts = [
    {"text": "The knight entered the hidden city."},
    {"text": "Legends spoke of ancient magic."},
]
dataset = Dataset.from_list(texts)

def tokenize(batch):
    out = tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    return out

tokenized = dataset.map(tokenize, batched=True)

def add_labels(batch):
    batch["labels"] = batch["input_ids"].copy()
    return batch

tokenized = tokenized.map(add_labels)

tokenized.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)

training_args = TrainingArguments(
    output_dir="./demo_model",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    learning_rate=2e-5,
    use_cpu=True, 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
)

trainer.train()

prompt = "Continue this fantasy story:\nA wandering knight entered the hidden city and discovered"

inputs = tokenizer(prompt, return_tensors="pt")

output = model.generate(
    **inputs,
    max_length=120,
    temperature=0.8,
    top_p=0.9
)

print(tokenizer.decode(output[0], skip_special_tokens=True))

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Step,Training Loss


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Continue this fantasy story:
A wandering knight entered the hidden city and discovered a mysterious, ancient artifact. This artifact had the power to grant wishes, but it also came with a price. The knight was intrigued, and he decided to make a wish.

He wished for the ability to heal any wound, to cure any disease, and to bring peace to the land. The artifact glowed brightly and granted his wish. However, with the power of the artifact came a curse. Every time he used it, he would lose a piece of himself.

As he journeyed on, the knight began to
